# Neural Network

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
from sklearn.neural_network import MLPClassifier
import pandas as pd
import seaborn as sns


In [ ]:
particle_df = pd.read_csv('data/training.csv')

In [ ]:
particle_df['Label'].unique()
particle_df.columns

In [ ]:
# 50 inputs, 6 outputs
model1 = MLPClassifier(hidden_layer_sizes=[20, 30, 40], max_iter=500)
model2 = MLPClassifier(hidden_layer_sizes=[30, 50, 30], max_iter=500)
model3 = MLPClassifier(hidden_layer_sizes=[30, 50, 30, 50], max_iter=500)
model4 = MLPClassifier(hidden_layer_sizes=[50, 30, 50, 30], max_iter=500)
model5 = MLPClassifier(hidden_layer_sizes=[50, 40, 40], max_iter=500)



In [ ]:
for i in range(1, 10, 2):
    df = particle_df.sample(frac=i*.02)
    X = df.drop(axis=1, columns=['Label'])
    y=df['Label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.2)
    results4 = cross_validate(model4, X_train, y_train, scoring='f1_macro')
    display(i*.02, results4)

In [ ]:
df = particle_df.sample(frac=.5)
X = df.drop(axis=1, columns=['Label'])
y=df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.2)

In [ ]:
results1 = cross_validate(model1, X_train, y_train, scoring='f1_macro', cv=3, n_jobs=-1)
results2 = cross_validate(model2, X_train, y_train, scoring='f1_macro', cv=3, n_jobs=-1)
results3 = cross_validate(model3, X_train, y_train, scoring='f1_macro', cv=3, n_jobs=-1)
results4 = cross_validate(model4, X_train, y_train, scoring='f1_macro', cv=3, n_jobs=-1)

In [ ]:
results5 = cross_validate(model5, X_train, y_train, scoring='f1_macro')
results5

In [ ]:
display(results1, results2, results3, results4, results5)

In [ ]:
# you may need to install networkx with pip
model5 = MLPClassifier(hidden_layer_sizes=[50, 40, 40], max_iter=500)

import networkx as nx
import colorsys
import matplotlib.pylab as plt

def show_ann(mlp):
    hidden_layers_n = len(mlp.coefs_)-1
    layers_n = hidden_layers_n + 2
    input_neurons_n = len(mlp.coefs_[0])
    hidden_neurons_n = [len(mlp.coefs_[i+1]) for i in range(hidden_layers_n)]
    output_neurons_n = len(mlp.coefs_[-1][0])

    G = nx.DiGraph()
    pos = {}

    # Create the neurons of the input layer
    for i in range(input_neurons_n):
        pos['Layer0_{}'.format(i)] = (i,layers_n-1)

    for j in range(hidden_layers_n):
        # Create the neurons of the j'th hidden layer
        prev_layer = j
        cur_layer = j+1
        if (j == 0):
            prev_size = input_neurons_n
        else:
            prev_size = hidden_neurons_n[j-1]
        for i in range(hidden_neurons_n[j]):
            pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
            for k in range(prev_size):
                w = mlp.coefs_[prev_layer][k][i]
                G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    # Create the neurons of the output layer
    prev_layer = hidden_layers_n
    cur_layer = hidden_layers_n+1
    for i in range(output_neurons_n):
        pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
        for k in range(hidden_neurons_n[-1]):
            w = mlp.coefs_[prev_layer][k][i]
            G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    edges = G.edges()
    colors = [colorsys.hsv_to_rgb(0 if G[u][v]['weight'] < 0 else 0.65,
                                  1,#min(1, abs(G[u][v]['weight'])),
                                  1) for u,v in edges]
    weights = [abs(G[u][v]['weight'])*2 for u,v in edges]
    plt.figure(figsize=(15, 18))
    nx.draw(G, pos, node_color='y', node_size=450, width=weights, edge_color=colors)
    
show_ann(model1.fit(X_train, y_train))

In [ ]:
show_ann(model3.fit(X_train, y_train))

In [ ]:
show_ann(model5.fit(X_train, y_train))

In [ ]:
pred = model5.fit(X_train, y_train).predict(X_test)


In [ ]:
pred[0:10]
y_test.value_counts()

In [ ]:
cm = confusion_matrix(y_test, pred, labels=['Electron', 'Ghost', 'Pion', 'Muon', 'Kaon', 'Proton'])
ConfusionMatrixDisplay(cm, display_labels=['Electron', 'Ghost', 'Pion', 'Muon', 'Kaon', 'Proton']).plot()
precision_recall_fscore_support(y_test, pred, labels=['Electron', 'Ghost', 'Pion', 'Muon', 'Kaon', 'Proton'])
f1_score(y_test, pred, average='macro')
